# Softmax_Logistic cho bài toán phân loại

In [3]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.neighbors import  KNeighborsRegressor
from umap_function import UMAP_Processor
from pca_function import PCA_Processor
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from data_preprocessor import DataPreprocessor
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [6]:
classifiers = [
    LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200),
]
class softmax_Logistic_Processor:
    def __init__(self, data_path, target_column, n_components=2, test_size=0.2, random_state=42):
        self.data_path = data_path
        self.target_column = target_column
        self.n_components = n_components
        self.test_size = test_size
        self.random_state = random_state

        preprocessor = DataPreprocessor(self.data_path, self.target_column)
        self.X, self.y, self.df = preprocessor.preprocess()

    def _scale_and_impute(self, X):
        scaler = StandardScaler()
        imputer = SimpleImputer(strategy='mean')
        return imputer.fit_transform(scaler.fit_transform(X))

    def _evaluate_classifiers(self, X_train, X_test, y_train, y_test, classifiers):
        results = {}
        for clf in classifiers:
            clf_name = clf.__class__.__name__
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            results[clf_name] = {
                "accuracy": accuracy_score(y_test, y_pred),
                "classification_report": classification_report(y_test, y_pred),
                "matrix": confusion_matrix(y_test, y_pred)
            }
        return results
    def results_classifier_original(self, classifiers):
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=self.test_size,
                                                            random_state=self.random_state)
        X_train_scaled, X_test_scaled = self._scale_and_impute(X_train), self._scale_and_impute(X_test)
        return self._evaluate_classifiers(X_train_scaled, X_test_scaled, y_train, y_test, classifiers)

    def apply_pca(self, classifiers):
        pca_processor = PCA_Processor(
            data_path='../data/framingham.csv',  # Đường dẫn đến file dữ liệu
            target_column='TenYearCHD',  # Cột mục tiêu
            n_components=5,  # Số chiều sau khi giảm chiều
            test_size=0.2,  # Tỷ lệ dữ liệu dùng để thử nghiệm
            random_state=42  # Hạt giống cho phép lặp lại các kết quả
        )

        pca_processor.n_components = 5

        results_pca_before = pca_processor.apply_pca_before_split(classifiers)
        print("\n\n===== Kết quả với dữ liệu sau khi áp dụng giảm chiều PCA =====")
        print_results(results_pca_before)

    def apply_umap(self, classifiers):
        umap_processor = UMAP_Processor(
            data_path = '../data/framingham.csv',
            target_column = 'TenYearCHD',
            n_components = 5,
            test_size = 0.2,  # Tỷ lệ dữ liệu dùng để thử nghiệm
            random_state = 42  # Hạt giống cho phép lặp lại các kết quả
        )
        # Kết quả sau khi áp dụng umap với n_components tối ưu
        results_after_umap = umap_processor.apply_optimal_n_components(classifiers)

        print("\n\n===== Kết quả với dữ liệu sau khi áp dụng giảm chiều UMAP =====")
        print_results(results_after_umap)

def print_results(results):
    for clf_name, metrics in results.items():
        print(f"Accuracy {clf_name:<25}{metrics['accuracy'] : <10.4f}")
        print(f"\nClassification Report for {clf_name}:")
        print(metrics['classification_report'])
        print(f"\nConfusion Matrix for {clf_name}:")
        print(metrics['matrix'])

In [7]:
softmax_logistic = softmax_Logistic_Processor(
    data_path='../data/framingham.csv',  
    target_column='TenYearCHD',  
    n_components=5,  
    test_size=0.2,
    random_state=42
)

In [8]:
print("\n\n===== Kết quả với dữ liệu gốc =====")
results_original = softmax_logistic.results_classifier_original(classifiers)
print_results(results_original)



===== Kết quả với dữ liệu gốc =====
Accuracy LogisticRegression       0.6539    

Classification Report for LogisticRegression:
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       735
           1       0.64      0.67      0.65       704

    accuracy                           0.65      1439
   macro avg       0.65      0.65      0.65      1439
weighted avg       0.65      0.65      0.65      1439


Confusion Matrix for LogisticRegression:
[[472 263]
 [235 469]]


In [9]:
softmax_logistic.apply_pca(classifiers)



===== Kết quả với dữ liệu sau khi áp dụng giảm chiều PCA =====
Accuracy LogisticRegression       0.6525    

Classification Report for LogisticRegression:
              precision    recall  f1-score   support

           0       0.67      0.64      0.65       735
           1       0.64      0.66      0.65       704

    accuracy                           0.65      1439
   macro avg       0.65      0.65      0.65      1439
weighted avg       0.65      0.65      0.65      1439


Confusion Matrix for LogisticRegression:
[[473 262]
 [238 466]]


In [10]:
softmax_logistic.apply_umap(classifiers)



===== Kết quả với dữ liệu sau khi áp dụng giảm chiều UMAP =====
Accuracy LogisticRegression       0.5490    

Classification Report for LogisticRegression:
              precision    recall  f1-score   support

           0       0.56      0.59      0.57       735
           1       0.54      0.51      0.53       704

    accuracy                           0.55      1439
   macro avg       0.55      0.55      0.55      1439
weighted avg       0.55      0.55      0.55      1439


Confusion Matrix for LogisticRegression:
[[430 305]
 [344 360]]


# Softmax_Logistic cho bài toán hồi quy

In [19]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

class RegressionProcessor:
    def __init__(self, data_path, target_column, test_size=0.2, random_state=42):
        self.data_path = data_path
        self.target_column = target_column
        self.test_size = test_size
        self.random_state = random_state

        preprocessor = DataPreprocessor(self.data_path, self.target_column)
        self.X, self.y = preprocessor.preprocess()

    def predict_probabilities(self):
        model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
        model.fit(self.X, self.y)
        probabilities = model.predict_proba(self.X)
        return probabilities

    def evaluate_regression(self):
        probabilities = self.predict_probabilities()
        decision_values = probabilities[:, 1]

        # Chia dữ liệu thành tập huấn luyện và kiểm tra
        X_train, X_test, y_train, y_test = train_test_split(self.X, decision_values, test_size=self.test_size,
                                                            random_state=self.random_state)

        # Hồi quy với Linear Regression
        reg_model1 = LinearRegression()
        reg_model1.fit(X_train, y_train)
        y_pred1 = reg_model1.predict(X_test)

        # Hồi quy với KNeighborsRegressor
        reg_model2 = KNeighborsRegressor()
        reg_model2.fit(X_train, y_train)
        y_pred2 = reg_model2.predict(X_test)

        # Đánh giá hiệu suất
        mse1 = mean_squared_error(y_test, y_pred1)
        r2_1 = r2_score(y_test, y_pred1)

        mse2 = mean_squared_error(y_test, y_pred2)
        r2_2 = r2_score(y_test, y_pred2)

        print(f'Mean Squared Error (Linear Regression): {mse1:.4f}, R^2 Score: {r2_1:.4f}')
        print(f'Mean Squared Error (KNN): {mse2:.4f}, R^2 Score: {r2_2:.4f}')

        results = {
            'Model': ['Linear Regression', 'KNN'],
            'MSE': [mse1,mse2],
            'R^2 Score': [r2_1,r2_2]
        }
        results_df = pd.DataFrame(results)
        return results_df

    def evaluate_regression_pca(self):
        # Giảm chiều dữ liệu bằng PCA
        pca = PCA(n_components=5)
        X_reduced = pca.fit_transform(self.X)

        # Chia dữ liệu đã giảm chiều thành tập huấn luyện và kiểm tra
        X_train, X_test, y_train, y_test = train_test_split(X_reduced, self.y, test_size=self.test_size,
                                                            random_state=self.random_state)
        # Hồi quy với Linear Regression
        reg_model1 = LinearRegression()
        reg_model1.fit(X_train, y_train)
        y_pred1 = reg_model1.predict(X_test)
    
        # Hồi quy với KNeighborsClassifier
        reg_model2 = KNeighborsClassifier()
        reg_model2.fit(X_train, y_train)
        y_pred2 = reg_model2.predict(X_test)
        
        mse1 = mean_squared_error(y_test, y_pred1)
        r2_1 = r2_score(y_test, y_pred1)
        
        # Đánh giá hiệu suất
        mse2 = mean_squared_error(y_test, y_pred2)
        r2_2 = r2_score(y_test, y_pred2)

        print(f'Mean Squared Error (PCA Linear Regression): {mse1:.4f}, R^2 Score: {r2_1:.4f}')
        print(f'Mean Squared Error (KNN): {mse2:.4f}, R^2 Score: {r2_2:.4f}')
        
        # Tạo DataFrame từ kết quả mô hình gốc
        results = {
            'Model': ['PCA Linear Regression', ' PCA KNN'],
            'MSE': [mse1,mse2],
            'R^2 Score': [r2_1,r2_2]
        }
        results_df = pd.DataFrame(results)
        return results_df

if __name__ == "__main__":
    reg_processor = RegressionProcessor(data_path='../data/framingham.csv', target_column='TenYearCHD')

    results_df = reg_processor.evaluate_regression()
    results_pca_df = reg_processor.evaluate_regression_pca()
    combined_df = pd.concat([results_df, results_pca_df], ignore_index=True)

    print("\nKết quả đánh giá hồi quy:")
    print(combined_df)

Mean Squared Error (Linear Regression): 0.0004, R^2 Score: 0.9883
Mean Squared Error (KNN): 0.0004, R^2 Score: 0.9883
Mean Squared Error (PCA Linear Regression): 0.2270, R^2 Score: 0.0917
Mean Squared Error (KNN): 0.2231, R^2 Score: 0.1073

Kết quả đánh giá hồi quy:
                   Model       MSE  R^2 Score
0      Linear Regression  0.000433   0.988299
1                    KNN  0.000433   0.988300
2  PCA Linear Regression  0.226976   0.091673
3                PCA KNN  0.223072   0.107299
